In [ ]:
import os
import numpy as np
import pandas as pd
import json

import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tqdm.notebook import tqdm

In [ ]:
df= pd.read_csv('/kaggle/input/vinbigdata-512-image-dataset/vinbigdata/train.csv')
df.image_id= '/kaggle/input/vinbigdata-512-image-dataset/vinbigdata/train/'+df.image_id+'.png'
df = df[df.class_id!=14].reset_index(drop = True)
df.head()

In [ ]:
df['x_min']= (df.x_min/df.width)*512
df['x_max']= (df.x_max/df.width)*512

df['y_min']= (df.y_min/df.height)*512
df['y_max']= (df.y_max/df.height)*512

In [ ]:
df['w']= df.x_max- df.x_min
df['h']= df.y_max- df.y_min
df_train= df[['image_id', 'x_min', 'y_min', 'w', 'h', 'class_id', 'class_name']].reset_index(drop=True)
df_train.head()

In [ ]:
df_train.isna().sum()

In [ ]:
image_ids = df_train['image_id'].unique()
image_dict = dict(zip(image_ids, range(len(image_ids))))
len(image_dict)

In [ ]:
json_dict = {"images": [], "type": "instances", "annotations": [], "categories": []}

In [ ]:
for image_id in image_ids:
    image = {'file_name': image_id, 
             'height': 512, 
             'width': 512, 
             'id': image_dict[image_id]}
    json_dict['images'].append(image)

In [ ]:
for name, cls_id in zip(df_train.class_name.unique(), df_train.class_id.unique()):
    print(name, cls_id)

In [ ]:
for name, cls_id in zip(df_train.class_name.unique(), df_train.class_id.unique()):
    categories = {'supercategory': 'master', 'id': cls_id, 'name': name}
    json_dict['categories'].append(categories)

# categories = {'supercategory': 'master', 'id': df_train.class_id.unique(), 'name': df_train.class_name.unique()}
# json_dict['categories'].append(categories)

In [ ]:
############################## Need HELP ##########################################
for idx, row in tqdm(df_train.iterrows()): 
    image_id = image_dict[row['image_id']]
    ann = {'area': row['w'] * row['h'], 
           'iscrowd': 0, 
           'image_id': image_id,                        
           'bbox': [row['x_min'], row['y_min'], row['w'], row['h']],
           'category_id': row.class_id, 
           'id': idx,
           'segmentation': []}
    
    json_dict['annotations'].append(ann)

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [ ]:
annFile='instances_Images.json'

json_fp = open(annFile, 'w',encoding='utf-8')
json_str = json.dumps(json_dict,cls=NpEncoder)
json_fp.write(json_str)
json_fp.close()

In [ ]:
!git clone https://github.com/kamauz/EfficientDet.git
#https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch.git

In [ ]:
%cd /kaggle/working/EfficientDet/

In [ ]:
#%%capture
!python setup.py build_ext --inplace

In [ ]:
!pip install -r requirements.txt

In [ ]:
from model import efficientdet
from losses import smooth_l1, focal
from efficientnet import BASE_WEIGHTS_PATH, WEIGHTS_HASHES
from generators.common import Generator

In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' -q

from pycocotools.coco import COCO

In [ ]:
def preprocess_image(image):
    image = image.astype(np.float32)
    image /= 255.0
#     mean = [0.485, 0.456, 0.406]
#     std = [0.229, 0.224, 0.225]
#     image -= mean
#     image /= std

    return image

In [ ]:
def postprocess_boxes(boxes, height, width):
    c_boxes = boxes.copy()
    c_boxes[:, 0] = np.clip(c_boxes[:, 0], 0, width - 1)
    c_boxes[:, 1] = np.clip(c_boxes[:, 1], 0, height - 1)
    c_boxes[:, 2] = np.clip(c_boxes[:, 2], 0, width - 1)
    c_boxes[:, 3] = np.clip(c_boxes[:, 3], 0, height - 1)
    return c_boxes

In [ ]:
class CocoGenerator(Generator):
    def __init__(self, data_dir, set_name, **kwargs):                                    
        self.coco = COCO('/kaggle/working/instances_Images.json')                
        self.image_ids = self.coco.getImgIds()
        self.load_classes()

        super(CocoGenerator, self).__init__(**kwargs)

    def load_classes(self): 
        categories = self.coco.loadCats(self.coco.getCatIds())
        categories.sort(key=lambda x: x['id'])

        self.classes = {}
        self.coco_labels = {}
        self.coco_labels_inverse = {}
        for c in categories:
            self.coco_labels[len(self.classes)] = c['id']
            self.coco_labels_inverse[c['id']] = len(self.classes)
            self.classes[c['name']] = len(self.classes)

        self.labels = {}
        for key, value in self.classes.items():
            self.labels[value] = key

    def size(self):
        return len(self.image_ids)

    def num_classes(self):
        return 1

    def has_label(self, label):
        return label in self.labels

    def has_name(self, name):
        return name in self.classes

    def name_to_label(self, name):
        return self.classes[name]

    def label_to_name(self, label):
        return self.labels[label]

    def coco_label_to_label(self, coco_label):
        return self.coco_labels_inverse[coco_label]

    def coco_label_to_name(self, coco_label):
        return self.label_to_name(self.coco_label_to_label(coco_label))

    def label_to_coco_label(self, label):
        return self.coco_labels[label]

    def image_aspect_ratio(self, image_index):
        image = self.coco.loadImgs(self.image_ids[image_index])[0]
        return float(image['width']) / float(image['height'])

    def load_image(self, image_index):        
        image_info = self.coco.loadImgs(self.image_ids[image_index])[0]    
        path = os.path.join('/kaggle/input/vinbigdata-512-image-dataset/vinbigdata/train/', image_info['file_name'])
        print(path)
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        image = preprocess_image(image)
        
        return image

    def load_annotations(self, image_index):
        annotations_ids = self.coco.getAnnIds(imgIds=self.image_ids[image_index], iscrowd=False)
        annotations = {'labels': np.empty((0,), dtype=np.float32), 'bboxes': np.empty((0, 4), dtype=np.float32)}

        if len(annotations_ids) == 0:
            return annotations

        coco_annotations = self.coco.loadAnns(annotations_ids)
        for idx, a in enumerate(coco_annotations):
            # some annotations have basically no width / height, skip them
            if a['bbox'][2] < 1 or a['bbox'][3] < 1:
                continue

            annotations['labels'] = np.concatenate(
                [annotations['labels'], [a['category_id'] - 1]], axis=0)
            annotations['bboxes'] = np.concatenate([annotations['bboxes'], [[
                a['bbox'][0],
                a['bbox'][1],
                a['bbox'][0] + a['bbox'][2],
                a['bbox'][1] + a['bbox'][3],
            ]]], axis=0)           

        return annotations

In [ ]:
phi = 4
score_threshold=0.4

In [ ]:
train_generator = CocoGenerator(data_dir=None, set_name=None, batch_size = 2, phi = phi)

In [ ]:
for jj in train_generator:
    break

In [ ]:
jj[0][0].shape, jj[1][0].shape#, jj[2][0].shape

In [ ]:
jj[0][0][0].mean()

In [ ]:
plt.imshow(jj[0][0][0])